In [8]:
from maskmm.ipstartup import *
import os
import glob
from os.path import join
import random
import torch
from torch import load

import skimage.io
from skimage.io import imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
import random

from maskmm.models.maskrcnn import MaskRCNN
from maskmm import learner
from maskmm.utils import visualize

from maskmm.datasets.nuke.config import Config
from maskmm.datasets.nuke.dataset import Dataset

from maskmm.mytools import *

ROOT_DIR = "/home/ubuntu/maskmm"
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "data/models/mask_rcnn_coco.pth")
DATA = join(expanduser("~"), "data", "nuke") 

class Config(Config):
    NAME = "mm"
    STEPS_PER_EPOCH = 1
    VALIDATION_STEPS = 1
    GPU_COUNT=1
    NPRANDOM = True
config = Config()
rngreset()

[root:INFO]:starting (cellevents.py:36, time=00:11)


time: 38.8 ms


In [9]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

[root:INFO]:starting (cellevents.py:36, time=00:12)


time: 1.69 ms


## Dataset

In [10]:
# create validation sample
pvalid = .2
trainpath = join(DATA, "stage1_train")
df = pd.DataFrame(os.listdir(trainpath), columns=["image"])
df["subset"] = np.random.random(len(df))>pvalid
df.loc[df.subset==True, "subset"] = "train"
df.loc[df.subset==False, "subset"] = "valid"

df.to_pickle(join(DATA, "subset.pkl"))
df.subset.value_counts()

[root:INFO]:starting (cellevents.py:36, time=00:12)


train    505
valid    123
Name: subset, dtype: int64

time: 16.9 ms


In [11]:
dataset_train = Dataset(config)
dataset_train.load_nuke(trainpath, "train")
dataset_train.prepare()

dataset_val = Dataset(config)
dataset_val.load_nuke(trainpath, "valid")
dataset_val.prepare()

[root:INFO]:starting (cellevents.py:36, time=00:12)


time: 43.8 ms


In [12]:
%%s
# Load and display random samples
image_ids = np.random.choice(dataset_train.image_ids, 4)
for image_id in image_ids:
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

[root:INFO]:starting (cellevents.py:36, time=00:12)


time: 1.2 ms


## Create Model

In [13]:
model = MaskRCNN(model_dir=MODEL_DIR, config=config).to(config.DEVICE)
model.initialize_weights()

# load pretrained except final layers that depend on NUM_CLASSES
params = torch.load(COCO_MODEL_PATH)
params.pop('classifier.linear_class.weight')
params.pop("classifier.linear_bbox.weight")
params.pop("mask.conv5.weight")
params.pop('classifier.linear_class.bias')
params.pop("classifier.linear_bbox.bias")
params.pop("mask.conv5.bias")
model.load_state_dict(params, strict=False)
_ = model.to(config.DEVICE)

[root:INFO]:starting (cellevents.py:36, time=00:12)


time: 318 ms


In [16]:
from maskmm.callbacks import Multiloss
from fastai import *

callbacks = [Multiloss]

# create and fit model
data = DataBunch.create(dataset_train, dataset_val, bs=1, num_workers=0)
learn = Learner(data, model, callbacks=None)
learn.fit(1)

[root:INFO]:starting (cellevents.py:36, time=00:14)


AttributeError: 'Dataset' object has no attribute 'loss_func'

time: 22.3 ms


# maskmm learner

In [18]:
# Create model in training mode
m = learner.Learner(model, dataset_train, dataset_val)
m.train(.01, 3, "heads")

[root:INFO]:starting (cellevents.py:36, time=12:33)



Starting at epoch 1. LR=0.01

Checkpoint Path: /home/ubuntu/maskmm/logs/mm20181023_1223/mask_rcnn_mm_{:04d}.pth
Epoch 1/3.


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

time: 842 ms


In [13]:
torch.tensor(config.MEAN_PIXEL, dtype=torch.float).type()

[root:INFO]:starting (cellevents.py:36, time=12:26)


'torch.FloatTensor'

time: 4.24 ms


In [ ]:
match("images"), match("gt_boxes")

In [ ]:
match("anchors_pre"), match("gt_boxes_pre"), match("overlaps")

In [ ]:
match("anchors_pre", tol=0), match("gt_boxes_pre", tol=0), match("overlaps", tol=0)

In [ ]:
match("test1"), match("test2"), match("test3")

In [ ]:
match("rpn_match"), match("rpn_bbox")

In [ ]:
match("pre_conv", tol=0), match("post_conv"), match("post_conv", "post_convtest")

In [ ]:
mse(load("inputs1"), load("inputs10"))

In [ ]:
match("inputs0"), match("inputs1"), match("normalized_boxes")

In [ ]:
load("masks").sum(),load("masks0").sum()

In [ ]:
mse(load("masks"),load("masks0"))

In [ ]:
match("proposals"), match("rois"), match("roi_gt_class_ids"), match("deltas"), match("masks")

# timings

In [ ]:
%%s
from maskmm.utils import box_utils
a=load("anchors_pre")
b=load("gt_boxes_pre")
%timeit a=load("anchors_pre")
%timeit b=load("gt_boxes_pre")
%timeit overlaps = box_utils.np_compute_overlaps(a,b)
save(overlaps.numpy(), "overlaps")
match("overlaps")

In [ ]:
%%s
from maskmm.utils import box_utils
a=torch.Tensor(load("anchors_pre")).cpu()
b=torch.Tensor(load("gt_boxes_pre")).cpu()
%timeit a=torch.Tensor(load("anchors_pre")).cpu()
%timeit b=torch.Tensor(load("gt_boxes_pre")).cpu()
%timeit overlaps = box_utils.torch_compute_overlaps(a,b)
save(overlaps.numpy(), "overlaps")
match("overlaps")

In [ ]:
%%s
from maskmm.utils import box_utils
a=torch.Tensor(load("anchors_pre")).cuda()
b=torch.Tensor(load("gt_boxes_pre")).cuda()
%timeit a=torch.Tensor(load("anchors_pre")).cuda()
%timeit b=torch.Tensor(load("gt_boxes_pre")).cuda()
%timeit overlaps = box_utils.torch_compute_overlaps(a,b)
save(overlaps.numpy(), "overlaps")
match("overlaps")